### 16/04/2025 AlgoForce Slippage Calculation Task

### What is slippage

🔁 What is Slippage?
Slippage = Execution Price - Expected Price

It can be:

- `Positive slippage`: You get a better price than expected.

- `Negative slippage`: You get a worse price than expected.

⚙️ Binance Futures Example
Let’s say you want to go long on BTC/USDT and use a market order:

Price shown: 65,000

But the actual order book has limited liquidity at 65K.

Your order ends up filled at 65,020.

That $20 difference is slippage.

Binance does show expected slippage warnings if it's likely to be large (especially on mobile).

### TODO:

1. Get a crypto pair that is highly volatile (For this code I will get NEAR_VET from AF5 account NEAR(long)/VET(short)) and put slippage data to json
2. Get the average/mean of the orders
3. Check if it is negative or positive pnl ?
4. percentage, min of the price(for long)|max(for short) and max and average
5. VWAP Formula for average percentage (slippage value - executed price)
6. Make excel and show graph

## 1. Get the data of NEAR_VET from a json file

In [3]:
import json

# Load the data from the VET_NEAR_slippage.json file
with open("VET_NEAR_slippage.json", "r") as file:
    data = json.load(file)

## 2. Put values into variables and output

In [4]:
# Create a summary dictionary
summary = {}

# Extract LONG summary
long_data = data["NEAR_VET"]["LONG"]
summary["LONG"] = {}
for key, value in long_data.items():
    if key.startswith("LONG"):
        summary["LONG"][key] = {
            "time": value["time"],
            "total_pnl": value["total_pnl"],
            "total_fee": value["total_fee"],
            "trading_total": value["trading_total"],
            "prices": value["prices"]
        }

# Extract SHORT summary
short_data = data["NEAR_VET"]["SHORT"]
summary["SHORT"] = {}
for key, value in short_data.items():
    if key.startswith("SHORT"):
        summary["SHORT"][key] = {
            "time": value["time"],
            "total_pnl": value["total_pnl"],
            "total_fee": value["total_fee"],
            "trading_total": value["trading_total"],
            "prices": value["prices"]
        }

# Construct the string summary
summary_string = f"Account: {data['NEAR_VET']['account']}\n"
summary_string += f"Type: {data['NEAR_VET']['type']}\n\n"

# Long Summary
for long_key, long_info in summary["LONG"].items():
    summary_string += f"{long_key}:\n"
    summary_string += f"  Time: {long_info['time']}\n"
    summary_string += f"  Total PnL: {long_info['total_pnl']}\n"
    summary_string += f"  Total Fee: {long_info['total_fee']}\n"
    summary_string += f"  Trading Total: {long_info['trading_total']}\n"
    
    # Include prices details
    summary_string += "  Prices:\n"
    for price in long_info["prices"]:
        summary_string += f"    Time: {price['time']}, Price: {price['trading_price']}, Executed: {price['executed']}, Fee: {price['fee']}, PnL: {price['pnl']}, Total: {price['total']}\n"
    
    summary_string += "\n"

# Print values
for short_key, short_info in summary["SHORT"].items():
    summary_string += f"{short_key}:\n"
    summary_string += f"  Time: {short_info['time']}\n"
    summary_string += f"  Total PnL: {short_info['total_pnl']}\n"
    summary_string += f"  Total Fee: {short_info['total_fee']}\n"
    summary_string += f"  Trading Total: {short_info['trading_total']}\n"
    
    # Include prices details
    summary_string += "  Prices:\n"
    for price in short_info["prices"]:
        summary_string += f"    Time: {price['time']}, Price: {price['trading_price']}, Executed: {price['executed']}, Fee: {price['fee']}, PnL: {price['pnl']}, Total: {price['total']}\n"
    
    summary_string += "\n"

# Print the summary string
print(summary_string)

Account: AF5
Type: close

LONG_1:
  Time: 2025-04-15 16:00:20
  Total PnL: 204.66221971
  Total Fee: 0.69486398
  Trading Total: 1389.727998
  Prices:
    Time: 2025-04-15 16:00:20, Price: 0.023948, Executed: 8110, Fee: 0.09710914, PnL: 28.60761948, Total: 194.21828
    Time: 2025-04-15 16:00:20, Price: 0.023948, Executed: 15750, Fee: 0.1885905, PnL: 55.55733747, Total: 377.181
    Time: 2025-04-15 16:00:20, Price: 0.023947, Executed: 15463, Fee: 0.18514623, PnL: 54.52949632, Total: 370.292461
    Time: 2025-04-15 16:00:20, Price: 0.023947, Executed: 383, Fee: 0.00458585, PnL: 1.35063034, Total: 9.171701
    Time: 2025-04-15 16:00:20, Price: 0.023947, Executed: 3014, Fee: 0.03608812, PnL: 10.62872029, Total: 72.176258
    Time: 2025-04-15 16:00:20, Price: 0.023947, Executed: 3200, Fee: 0.0383152, PnL: 11.28463999, Total: 76.6304
    Time: 2025-04-15 16:00:20, Price: 0.023946, Executed: 12113, Fee: 0.14502894, PnL: 42.70377582, Total: 290.057898

LONG_2:
  Time: 2025-04-15 16:00:22
  To

## 3. Make slippage calculation

In [18]:
import json
import statistics

# Function to calculate the slippage as a percentage
def calculate_slippage(base_price, current_price, position_type):
    if position_type == 'LONG' and base_price >= current_price:
        return ((base_price - current_price) / base_price) * 100
    elif position_type == 'SHORT' and base_price <= current_price:
        return -((base_price - current_price) / base_price) * 100
    else:
        return -((base_price - current_price) / base_price) * 100

# Variables to accumulate total slippage, PnL, total fee, and total for LONG and SHORT
total_long_slippage = 0
total_short_slippage = 0
total_long_pnl = 0
total_short_pnl = 0
total_long_fee = 0  
total_short_fee = 0  
total_long_total = 0
total_short_total = 0

long_base_price = 0
short_base_price = 0

# Lists to store slippage values for calculating the mean (avoiding the first base price)
long_slippage_values = []
short_slippage_values = []
long_prices = []
short_prices = []

pair_name = ""
long_symbol = ""
short_symbol = ""

# Process LONG trades (VETUSDT)
for long_key, trade in data["NEAR_VET"]["LONG"].items():
    if long_key != "symbol" and 'prices' in trade:
        long_slippage_count = 0 
        # Make entry price the basis of all prices
        if long_key == "LONG_1" and long_slippage_count == 0:
            long_base_price = trade['prices'][0]['trading_price']  # First price as base for this position
        
        long_symbol = data["NEAR_VET"]["LONG"]["symbol"]  # Access symbol from the parent dictionary

        print(f"\nSlippage calculations for {long_key} ({long_symbol}):")
        
        # If skip_first_price is True, skip the first price for LONG_1
        for idx, price_data in enumerate(trade['prices']):
            current_price = price_data['trading_price']
            
            long_prices.append(current_price)
            
            if long_key == "LONG_1" and long_slippage_count == 0:
                print(f"  {long_slippage_count}: {price_data['time']} at {current_price}: EXECUTED PRICE")
                pair_name = f"{long_symbol}_"
                long_slippage_count += 1
                continue  # Skip the first price (base price)
            
            slippage = calculate_slippage(long_base_price, current_price, 'LONG')
            print(f"  {long_slippage_count}: {price_data['time']} at {current_price}: {slippage:.4f}%")
            
            # Store slippage values for calculating the mean
            long_slippage_values.append(slippage)
            long_slippage_count += 1
        
        # Add directly from the JSON data
        total_long_pnl += trade['total_pnl']
        total_long_fee += trade['total_fee']  # Sum the fees
        total_long_total += trade['trading_total']  # Sum the trading totals

# Process SHORT trades (NEARUSDT)
for short_key, trade in data["NEAR_VET"]["SHORT"].items():
    if short_key != "symbol" and 'prices' in trade:
        short_slippage_count = 0
        
        if short_key == "SHORT_1" and short_slippage_count == 0:
            short_base_price = trade['prices'][0]['trading_price']  # First price as base for this position

        short_symbol = data["NEAR_VET"]["SHORT"]["symbol"]  # Access symbol from the parent dictionary
        print(f"\nSlippage calculations for {short_key} ({short_symbol}):")
        
        # If skip_first_price is True, skip the first price for SHORT_1
        for idx, price_data in enumerate(trade['prices']):
            current_price = price_data['trading_price']
            
            short_prices.append(current_price)
            
            if short_key == "SHORT_1" and short_slippage_count == 0:
                print(f"  {short_slippage_count}: {price_data['time']} at {current_price}: EXECUTED PRICE")
                pair_name = f"{pair_name + short_symbol}"
                short_slippage_count += 1
                continue
            
            slippage = calculate_slippage(short_base_price, current_price, 'SHORT')
            print(f"  {short_slippage_count}: {price_data['time']} at {current_price}: {slippage:.4f}%")
                
            # Store slippage values for calculating the mean
            short_slippage_values.append(slippage)
        
        # Add directly from the JSON data
        total_short_pnl += trade['total_pnl']
        total_short_fee += trade['total_fee']  # Sum the fees
        total_short_total += trade['trading_total']  # Sum the trading totals
        
        short_slippage_values.append(slippage)

# Calculate the total fees for the pair
total_pair_fee = total_long_fee + total_short_fee

# Calculate the total trading price for LONG and SHORT
total_long_price = total_long_total  # Total price for LONG positions (no need for mean)
total_short_price = total_short_total  # Total price for SHORT positions (no need for mean)

# Calculate the combined total price
combined_total_price = total_long_price + total_short_price

# Calculate Slippage
long_slippage = -(statistics.mean(long_prices) - long_base_price)
short_slippage = statistics.mean(short_prices) - short_base_price

precision = max(2, len(str(total_short_pnl).split('.')[-1]))

# Print the final statistics with total prices instead of mean
print("\n--- STATISTICS ---")
print(f"\nLONG {long_symbol} STATS")
print(f"Slippage: {long_slippage:.{precision}f}%")  # Sum of all slippage values
print(f"Highest price: {max(long_prices)}")
print(f"Lowest price: {min(long_prices)}")
print(f"Total PnL: {total_long_pnl:.{precision}f}%")
print(f"Total fee: {total_long_fee:.{precision}f}")
print(f"Trade Total: {total_long_price:.{precision}f}")  # Total Price for LONG

print(f"\nSHORT {short_symbol} STATS")
print(f"Slippage: {short_slippage:.{precision}f}%")  # Sum of all slippage values
print(f"Highest price: {max(short_prices)}")
print(f"Lowest price: {min(short_prices)}")
print(f"Total PnL: {total_short_pnl:.{precision}f}%")
print(f"Total fee: {total_short_fee:.{precision}f}")
print(f"Trade Total: {total_short_price:.{precision}f}")  # Total Price for SHORT

print(f"\nCOMBINED {pair_name} STATS")
print(f"Combined slippage: {long_slippage + short_slippage:.{precision}f}%")  # Combined slippage sum
print(f"Combined Total PnL: {total_long_pnl + total_short_pnl:.{precision}f}%")
print(f"Combined Total fee: {total_pair_fee:.{precision}f}")
print(f"Combined Trade Total: {combined_total_price:.{precision}f}")  # Combined Total Price



Slippage calculations for LONG_1 (VETUSDT):
  0: 2025-04-15 16:00:20 at 0.023948: EXECUTED PRICE
  1: 2025-04-15 16:00:20 at 0.023948: 0.0000%
  2: 2025-04-15 16:00:20 at 0.023947: 0.0042%
  3: 2025-04-15 16:00:20 at 0.023947: 0.0042%
  4: 2025-04-15 16:00:20 at 0.023947: 0.0042%
  5: 2025-04-15 16:00:20 at 0.023947: 0.0042%
  6: 2025-04-15 16:00:20 at 0.023946: 0.0084%

Slippage calculations for LONG_2 (VETUSDT):
  0: 2025-04-15 16:00:22 at 0.023939: 0.0376%
  1: 2025-04-15 16:00:22 at 0.023938: 0.0418%
  2: 2025-04-15 16:00:22 at 0.023938: 0.0418%
  3: 2025-04-15 16:00:22 at 0.023937: 0.0459%
  4: 2025-04-15 16:00:22 at 0.023936: 0.0501%
  5: 2025-04-15 16:00:22 at 0.023936: 0.0501%
  6: 2025-04-15 16:00:22 at 0.023936: 0.0501%
  7: 2025-04-15 16:00:22 at 0.023933: 0.0626%
  8: 2025-04-15 16:00:22 at 0.023933: 0.0626%

Slippage calculations for LONG_3 (VETUSDT):
  0: 2025-04-15 16:00:23 at 0.02394: 0.0334%
  1: 2025-04-15 16:00:23 at 0.023939: 0.0376%
  2: 2025-04-15 16:00:23 at 0.0

In [20]:
import json
import pandas as pd
import os

# Path to the JSON file
data = 'VET_NEAR_slippage.json'  # Replace with your actual file name

# Define the Excel file path
excel_file = 'slippage_data.xlsx'
pair = 'NEARVET'

# Initialize the dictionary for storing log data
log_data = {}

# Combine the statistics into the log_data dictionary
log_data[pair] = pair
log_data['LONG Slippage'] = f"{long_slippage:.{precision}f}%"
log_data['LONG PnL'] = total_long_pnl
log_data['LONG Fee'] = total_long_fee
log_data['LONG Price'] = total_long_total

log_data['SHORT Slippage'] = f"{short_slippage:.{precision}f}%"
log_data['SHORT PnL'] = total_short_pnl
log_data['SHORT Fee'] = total_short_fee
log_data['SHORT Price'] = total_short_total

log_data['Pair Slippage'] = f"{short_slippage + long_slippage:.{precision}f}%"
log_data['Pair PnL'] = total_long_pnl + total_short_pnl
log_data['Pair Fee'] = total_long_fee + total_short_fee
log_data['Pair Price'] = total_long_total + total_short_total

# Convert the dictionary to a DataFrame
df = pd.DataFrame([log_data])

# Check if the Excel file exists and update or create new
if os.path.exists(excel_file):
    existing_df = pd.read_excel(excel_file, engine='openpyxl')
    updated_df = pd.concat([existing_df, df], ignore_index=True)
else:
    updated_df = df

# Save to the Excel file
updated_df.to_excel(excel_file, index=False, engine='openpyxl')

print(f"Data logged successfully to {excel_file}")


Data logged successfully to slippage_data.xlsx
